In [3]:
import numpy as np
import pandas as pd
from copy import deepcopy
from collections import Counter
from datetime import datetime
import json

In [178]:
class basic_preprocessing:
    
    
    def __init__(self, game_result, unit_result):

        self.game_result = game_result
        self.unit_result = unit_result
        self.augment_result = pd.read_csv('augment_list.csv', encoding='cp949')
        self.champion_result = pd.read_csv('champion_list.csv', encoding='cp949')
        self.item_result = pd.read_csv('item_list.csv', encoding='cp949')
        self.num_games = self.game_result.shape[0]//8        
        self.after_join = pd.merge(self.unit_result, self.game_result[['Datetime', 'Game_id', 'Player_id', 'Placement', 'augment1', 'augment2', 'augment3']], on=['Game_id', 'Player_id', 'Datetime'])
        self.after_join = self.after_join[['Datetime', 'Game_id', 'Player_id', 'Placement', 'augment1', 'augment2', 'augment3', 'champ_name', 'rarity', 'tier', 'item1', 'item2', 'item3']]


    
    def change_gloves_item(self):
    
        check = np.logical_or(self.after_join.item1=='도적의 장갑', self.after_join.item1=='장난꾸러기의 장갑')

        self.after_join.loc[check, 'item2']='None'

        self.after_join.loc[check, 'item3'] = 'None'

        return self.after_join
    
    
    
    
    def check_item_unit_itemver(self):
    
        check = self.after_join.apply(lambda x : 3-list(x).count('None'), axis=1)


        self.after_join['num_item'] = check

        return self.after_join
    

    
    
    
    
    #basic preprocessing
    
    
    
    def make_augment_stack(self, game_result, augment_list, num_game):

        check = game_result[['Datetime', 'Game_id', 'Player_id', 'Placement', 'augment1', 'augment2', 'augment3']].drop_duplicates()

        result = augment_list.copy()

        for i in range(1, 9):
            place = check.loc[check.Placement==i, ]

            augments = place.augment1.tolist() + place.augment2.tolist() + place.augment3.tolist()

            augments = Counter(augments)
            
            augment_result = [0]*7
            

            for augment in augments.keys():

                number = augments[augment]

                if i==1:
                    augment_score = [number, number*i, number*i, number, number, number, number]

                elif i<=4:
                    augment_score = [number, number*i, number*i, 0, 0, number, number]

                else:
                    augment_score = [number, number*i, 0, 0, 0, 0, 0]
                

                result.loc[result.Name == augment, 
                                 ['Count', 'Ave_score', 'Ave_score_save', 'Win_count', 'Win_rate', 'Save_count', 'Save_rate']] +=augment_score



        result.Ave_score = round(result.Ave_score/result.Count, 1)
        
        result.Ave_score_save = round(result.Ave_score_save/result.Save_count, 1)

        result.Win_rate = round(result.Win_rate/result.Count*100, 1)

        result.Save_rate = round(result.Save_rate/result.Count*100, 1)



        return result


    
    
    def make_unit_stack(self, game_result, champion_list, num_game):

        result = champion_list.copy()

        for placement in range(1, 9):

            for tier in range(1, 4):

                for num_item in range(1, 4):


                    case = np.logical_and(np.logical_and(game_result.Placement == placement, game_result.tier==tier),
                                          game_result.num_item == num_item)

                    champions = game_result.loc[case, 'champ_name'].tolist()

                    champions = Counter(champions)


                    for champion in champions.keys():

                        number = champions[champion]

                        if placement==1:



                            score = [number, number*placement, number*placement, number*tier, number*tier, number*tier, number, number, number, number, num_item*number, num_item*number, num_item*number, num_item*number]
                            
                            if num_item==3:
                                score += [number]*4
                            else:
                                score += [0]*4

                        elif placement<=4:

                            score = [number, number*placement, number*placement, number*tier, number*tier, 0, 0, 0, number, number, num_item*number, num_item*number, num_item*number, 0]
                            
                            if num_item==3:
                                score += [number]*3 +[0]
                                
                            else:
                                score += [0]*4
                        else:

                            score = [number, number*placement, 0, number*tier, 0, 0, 0, 0, 0, 0, num_item*number, num_item*number, 0, 0]

                            if num_item==3:

                                score += [number]*2 +[0]*2
                            else:
                                score += [0]*4


                        result.loc[result.Name == champion, ['Count', 'Ave_score', 'Ave_score_save', 'Ave_star', 'Ave_star_save', 'Ave_star_win',
                                                                          'Win_count', 'Win_rate', 'Save_count', 'Save_rate',
                                                                          'Num_items_count', 'Num_items_avg', 'Num_items_save', 'Num_items_win', 'Full_item_count', 'Full_item_rate', 'Full_item_save', 'Full_item_win']] += score


        result.Ave_score = round(result.Ave_score / result.Count, 1)
        
        result.Ave_score_save =round(result.Ave_score_save / result.Save_count, 1)

        result.Win_rate = round(result.Win_rate / result.Count*100, 1)

        result.Save_rate = round(result.Save_rate / result.Count*100, 1)
        
        result.Ave_star = round(result.Ave_star / result.Count, 1)
        
        result.Ave_star_save = round(result.Ave_star_save / result.Save_count, 1)
        
        result.Ave_star_win = round(result.Ave_star_win/result.Win_count, 1)


        result.Num_items_avg = round(result.Num_items_avg/result.Count, 1)
        
        result.Num_items_save = round(result.Num_items_save/result.Save_count, 1)
        
        result.Num_items_win = round(result.Num_items_win/result.Win_count, 1)

        result.Full_item_rate = round(result.Full_item_rate/result.Count*100, 1)
        
        result.Full_item_save = round(result.Full_item_save/result.Save_count*100, 1)
        
        result.Full_item_win = round(result.Full_item_win / result.Win_count*100, 1)



        return result
    
    
    
    
    
    
    
    def make_item_stack(self, game_result, item_list, num_game):
    
        result = item_list.copy()

        for placement in range(1, 9):

            check = game_result.loc[game_result.Placement==placement, :]

            items = check.item1.tolist() + check.item2.tolist() + check.item3.tolist()

            items = Counter(items)

            for item in items.keys():

                if item=='None':
                    continue

                else:

                    number = items[item]

                    if placement==1:
                        item_score = [number, number*placement, number*placement, number, number, number, number]

                    elif placement<=4:
                        item_score = [number, number*placement, number*placement, 0, 0, number, number]

                    else:
                        item_score = [number, number*placement, 0, 0, 0, 0, 0]

                result.loc[result.Name==item, ['Count', 'Ave_score', 'Ave_score_save', 'Win_count', 'Win_rate', 'Save_count', 'Save_rate']] += item_score


        result.Ave_score = round(result.Ave_score/result.Count, 1)
        
        result.Ave_score_save = round(result.Ave_score_save/result.Save_count, 1)

        result.Win_rate = round(result.Win_rate/result.Count*100, 1)

        result.Save_rate = round(result.Save_rate/result.Count*100, 1)


        return result

    
    
    
    
    
    
    
    # 증강체 기준 비교하기
    
    
    def augment_augment_crosscheck(self, augment):

        check = np.logical_or(np.logical_or(self.after_join.augment1==augment, self.after_join.augment2==augment), self.after_join.augment3==augment)

        check = self.after_join[check]
        
        num_game = len(check.Game_id.unique())

        check_augment = self.make_augment_stack(check, self.augment_result,num_game)

        return check_augment
    
    
    def augment_champion_crosscheck(self, augment):

        check = np.logical_or(np.logical_or(self.after_join.augment1==augment, self.after_join.augment2==augment), self.after_join.augment3==augment)

        check = self.after_join[check]
        
        num_game = len(check.Game_id.unique())

        check_champion = self.make_unit_stack(check, self.champion_result, num_game)


        return check_champion
    
    
    
    def augment_item_crosscheck(self, augment):

        check = np.logical_or(np.logical_or(self.after_join.augment1==augment, self.after_join.augment2==augment), self.after_join.augment3==augment)

        check = self.after_join[check]
        
        num_game = len(check.Game_id.unique())

        check_item = self.make_item_stack(check, self.item_result, num_game)

        return check_item
    
    
    # 챔피언 기준 비교하기
    
    
    def champion_champion_crosscheck(self, champion):
    
        check = self.after_join.loc[self.after_join.champ_name == champion, ['Game_id', 'Player_id']]

        check = pd.merge(self.after_join, check, on=['Game_id', 'Player_id'])
        
        num_game = len(check.Game_id.unique())

        check_champion = self.make_unit_stack(check, self.champion_result, num_game)

        return check_champion
    
    
    
    
    def champion_augment_crosscheck(self, champion):

        check = self.after_join.champ_name==champion

        check = self.after_join[check]
        
        num_game = len(check.Game_id.unique())

        check_augment = self.make_augment_stack(check, self.augment_result, num_game)

        return check_augment
    
    
    def champion_item_crosscheck(self, champion):
    
        check = self.after_join.champ_name==champion

        check = self.after_join[check]
        
        num_game = len(check.Game_id.unique())

        check_item = self.make_item_stack(check, self.item_result, num_game)

        return check_item
    
    
    
    # 아이템 기준
    
    
    def item_item_crosscheck(self, item):
    
        check = np.logical_or(np.logical_or(self.after_join.item1==item, self.after_join.item2==item), self.after_join.item3==item)


        check = self.after_join[check]
        
        num_game = len(check.Game_id.unique())


        check_item = self.make_item_stack(check, self.item_result, num_game)

        return check_item

    
    
    def item_augment_crosscheck(self, item):
    
        check = np.logical_or(np.logical_or(self.after_join.item1==item, self.after_join.item2==item), self.after_join.item3==item)


        check = self.after_join[check]
        
        num_game = len(check.Game_id.unique())

        check_augment = self.make_augment_stack(check, self.augment_result, num_game)

        return check_augment
    
    
    
    def item_champion_crosscheck(self, item):
    
        check = np.logical_or(np.logical_or(self.after_join.item1==item, self.after_join.item2==item), self.after_join.item3==item)    

        check = self.after_join[check]
        
        num_game = len(check.Game_id.unique())

        check_champion = self.make_unit_stack(check, self.champion_result, num_game)

        return check_champion
    
    
    
    def champion_summary(self, champion):
        
        check = self.after_join[self.after_join.champ_name==champion].copy()
        
        check.loc[:, 'Winner'] = check.Placement==1
        check.loc[:, 'Save'] = check.Placement<=4
        
        check = check.groupby('Datetime')
        
        result = check.agg({'rarity' : ['count'], 'Placement' : ['mean'], 'tier' : ['mean'], 'Save' : ['mean'], 'Winner' : ['mean']})
        
        result.columns = ['Count', 'Avg_placement', 'Avg_tier', 'Avg_save', 'Avg_winner']
        
        result = result.reset_index()
        
        return result
        
    
    
    def augment_summary(self, augment):
        
        check = np.logical_or(np.logical_or(self.after_join.augment1==augment, self.after_join.augment2==augment), self.after_join.augment3==augment)
        
        check = self.after_join[check].copy()
        
        check.loc[:, 'Winner'] = check.Placement==1
        check.loc[:, 'Save'] = check.Placement<=4
        
        check = check.groupby('Datetime')
        
        result = check.agg({'rarity' : ['count'], 'Placement' : ['mean'], 'Save' : ['mean'], 'Winner' : ['mean']})
        
        result.columns = ['Count', 'Avg_placement', 'Ave_save', 'Ave_winner']
        
        result = result.reset_index()
        
        return result
    
    
    
    def item_summary(self, item):
        
        check = np.logical_or(np.logical_or(self.after_join.item1==item, self.after_join.item2==item), self.after_join.item3==item)
        check = self.after_join[check].copy()
        
        check.loc[:, 'Winner'] = check.Placement==1
        check.loc[:, 'Save'] = check.Placement<=4
        
        check = check.groupby('Datetime')
        
        result = check.agg({'rarity' : ['count'], 'Placement' : ['mean'], 'Save' : ['mean'], 'Winner' : ['mean']})
        
        result.columns = ['Count', 'Avg_placement', 'Ave_save', 'Ave_winner']
        
        result = result.reset_index()
        
        return result        
        
    
    
    
    
    def making_champ_dictionary(self, champion):
        
        champ_summary = self.champion_summary(champion)
        champ_augment = self.champion_augment_crosscheck(champion)
        champ_champion = self.champion_champion_crosscheck(champion)
        champ_item = self.champion_item_crosscheck(champion)
        
        result = dict(summary = champ_summary.to_dict(), augment = champ_augment.to_dict(), champion = champ_champion.to_dict(),
                     item = champ_item.to_dict())
        
        return result
    
    
    
    
    def making_augment_dictionary(self, augment):
        
        aug_summary = self.augment_summary(augment)
        aug_augment = self.augment_augment_crosscheck(augment)
        aug_champion = self.augment_champion_crosscheck(augment)
        aug_item = self.augment_item_crosscheck(augment)
        
        result = dict(summary = aug_summary.to_dict(), augment = aug_augment.to_dict(), champion = aug_champion.to_dict(),
                     item = aug_item.to_dict())
        
        return result
    
    
    
    def making_item_dictionary(self, item):
        
        item_summary = self.item_summary(item)
        item_augment = self.item_augment_crosscheck(item)
        item_champion = self.item_champion_crosscheck(item)
        item_item = self.item_item_crosscheck(item)
        
        result = dict(summary = item_summary.to_dict(), augment = item_augment.to_dict(), champion = item_champion.to_dict(),
                     item = item_item.to_dict())
        
        return result
    
    
    
    def making_dictionary_automation(self):
        
        champ_list = self.champion_result.Name
        item_list = self.item_result.Name
        augment_list = self.augment_result.Name
        
        
        champ_dict = {i : self.making_champ_dictionary(i) for i in champ_list}
        item_dict = {i : self.making_item_dictionary(i) for i in item_list}
        augment_dict = {i : self.making_augment_dictionary(i) for i in augment_list}
        
        final_dictionary = dict(champion = champ_dict, item = item_dict, augment = augment_dict)
        
        return final_dictionary
        
        
    
    